In [2]:
import sys
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np

#make sure that opencv optimisation is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)
    
cv.useOptimized()

True

1. convert the code chunk food under section Divide an image into smaller patches using cropping into a function with the following signature:
crop_grid(img, num_horizonta_grid, num_vertical_grid, line_color)

In [40]:
#QUESTION 1 - FUNCTION
#img is the source image 
#num_horizontal_grid and num_vertical_grid are the number of patches along x and y axes. 
#line_color is the color of the grid line 
#the output of the function should be image with grids
def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    
    img_copy = img.copy()
    #this will return the height and width of the picture
    height, width = img.shape[:2]
        
    #M and N are basically number of pixels per patch
    M, N = int(height/num_horizontal_grid), int(width/num_vertical_grid)
    
    x1, y1 = 0, 0
    
    for y in range(0, height, M):
        for x in range(0, width, N):
            
            y1 = y + M    
            x1 = x + N

            if x1 >= width and y1 >= height:
                x1 = width - 1
                y1 = height - 1
                tile = img[y:height, x:width]
                #255 (0,255,0) means that green line
                #the 1 at the end is width of the line
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
                #cv.imshow('tile', tile)
            # When patch lower right y-coordinate exceeds patch height
            elif y1 >= height:
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            # When patch lower right x-coordinate exceeds patch width
            elif x1 >= width:
                x1 = width - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

            else:
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

    return img_copy

In [41]:
#QUESTION 1 - TEST THE FUNCTION
img = cv.imread("dog.jfif")

#line color is correspond to (B,G,R)
img2 = crop_grid(img, 2, 2, (255, 255, 255))

cv.imshow("inverse mask",img2)
cv.waitKey(0)
cv.destroyAllWindows()

2. Display image sequences of smooth transition of two images with different value of α. Refer to code in section "image blending". use "lena.jfif" and "coins.jfif" as the base images.

In [7]:
#QUESTION 2
img1 = cv.imread("lena.jfif")
img2 = cv.imread("coins.jfif")

#Resize img2
new_shape = img1.shape[:2]
img2 = cv.resize(img2, new_shape)

dst1 = cv.addWeighted(img1,0.75,img2 ,0.25,0)
dst2 = cv.addWeighted(img1,0.5,img2 ,0.25,0)
dst3 = cv.addWeighted(img1,0.25,img2 ,0.25,0)

cv.imshow("lena_base",img1)
cv.imshow("coin_base",img2)
cv.imshow("0.75α", dst1)
cv.imshow("0.50α", dst2)
cv.imshow("0.25α", dst3)

cv.waitKey(0)
cv.destroyAllWindows()

3. Rotate image by 45 degrees without croppping the sides of the image (Hint: There are 2 strategies to tackle these problems). use "lena.jfif" as the input image.
- use external libraries `imutils`.
-modify the transformation matrix.

In [29]:
#QUESTION 3
#Method 1
import imutils as im

img = cv.imread("lena.jfif")

#first parameter is the source image
#second parameter is the angle to rotate
rotate = im.rotate_bound(img, 45)

cv.imshow("result",rotate)

cv.waitKey(0)
cv.destroyAllWindows()

In [36]:
#QUESTION 3
#Method 2
import math

def rotate_about_center(src, angle, scale):
    
    width = src.shape[1] #take the width value of the source image
    height = src.shape[0] #take the height value of the source image
    
    radian = np.deg2rad(angle)  # covert the angle into radian
    
    # calculate the width and height of the new image
    newWidth = (abs(np.sin(radian)*height) + abs(np.cos(radian)*width))*scale
    newHeight = (abs(np.cos(radian)*height) + abs(np.sin(radian)*width))*scale
    
    #use getRotationMatrix2D function to get the rotation matrix
    rotation_matrix = cv.getRotationMatrix2D((newWidth*0.5, newHeight*0.5), angle, scale)

    
    # calculate to get the new rotation and center
    rotation_move = np.dot(rotation_matrix, np.array([(newWidth-width)*0.5, (newHeight-height)*0.5,0]))
   
    # the move only affects the translation, so update the translation
    # part of the transform
    rotation_matrix[0,2] += rotation_move[0]
    rotation_matrix[1,2] += rotation_move[1]
    
    return cv.warpAffine(src, rotation_matrix, (int(math.ceil(newWidth)), int(math.ceil(newHeight))), flags=cv.INTER_LANCZOS4)

In [37]:
#QUESTION 3
#METHOD 2

img = cv.imread("lena.jfif")

#the first parameter is the source of the image
#the second parameter is the degree of the rotation
#the third parameter is the scale of the image
rotated_image = rotate_about_center(img,45,1)

cv.imshow("result",rotated_image)

cv.waitKey(0)
cv.destroyAllWindows()

4. Use the images with titles: _"flower.jfif"_ and *"native-bee.png"*. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation](https://docs.opencv.org/4.x/d2/de8/group__core__array.html#ga60b4d04b251ba5eb1392c34425497e14) for more info. The result should resemble the following:  
![bee and flowers](image_embed/activity3.PNG "bee_flower")

In [39]:
#QUESTION 4
#load two image
lebah = cv.imread("native-bee.png")
bunga = cv.imread("flower.jfif")

#get the roi of the flower
#create roi
roi = lebah[0:bunga.shape[0], 0:bunga.shape[1]]

# Create mask of flower and its inverse mask 
flower_gray = cv.cvtColor(bunga,cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(flower_gray, 150, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)

# Black out the mask of flower
lebah_belakang = cv.bitwise_and(roi,roi,mask = mask_inv)

# Take region of flower
bunga_depan = cv.bitwise_and(bunga,bunga,mask = mask)

# Put flower in ROI and modify the bee image
dst = cv.add(lebah_belakang,bunga_depan)
lebah[0:bunga.shape[0], 0:bunga.shape[1]] = dst

cv.imshow('output',lebah)
cv.waitKey(0)
cv.destroyAllWindows()